
# Incorporating OLI Calculations with WaterTAP

#### Contact: Paul Vecchiarelli (paul.vecchiarelli@nrel.gov)

This tutorial will demonstrate basic usage of OLI Cloud calls using our custom API tools.

## Rationale

 - Simulations for realistic water sources are mathematically complex: 
 > $ Interactions \ge Cations * Anions$
 - OLI improves WaterTAP approximations and offloads computational resources

## Required OLI API Inputs


 - State variables (solute concentrations, temperature, pressure), which can beextracted from a state block
 
 - Login credentials
 
 - A chemistry (*.dbs) file
     - establishes state variables, phases, etc. to be considered in flash calls

In [ ]:
# used to specify state/convert variables
from pyomo.environ import units as pyunits

# used to build survey lists
from numpy import linspace

# used to execute OLI Cloud functions
from watertap.tools.oli_api.flash import Flash
from watertap.tools.oli_api.credentials import CredentialManager
from watertap.tools.oli_api.client import OLIApi

# 1. Specify State Variables.

- This data is used to construct inputs to OLI Cloud

In [ ]:
state_vars = {
    "temperature": 273.15,
    "pressure": 101325,
    "components": {
        "Na_+": 1120,
        "K_+": 15,
        "Ca_2+": 150,
        "Mg_2+": 33,
        "Cl_-": 1750,
        "SO4_2-": 260,
        "HCO3_-": 250,
        "SiO2": 30.5,
        "Ca[OH]2": 0,
        "Mg[OH]2": 0,
        "Na2CO3": 0,
    },
    "units": {
        "temperature": pyunits.K,
        "pressure": pyunits.Pa,
        "components": pyunits.mg / pyunits.L,
    },
}

# 2. Initialize Flash Instance.

 - We will run most of our methods with this class

In [ ]:
f = Flash()

# 3. Get Survey Parameters.

 - In this example, we will generate a temperature sweep survey

In [ ]:
survey = f.build_survey(
    {"Ca[OH]2": linspace(0, 1e3, 2),
     "Mg[OH]2": linspace(0, 1e3, 2),
     "Na2CO3": linspace(0, 1e3, 2),
     "Temperature": linspace(0, 100, 2),
     "Pressure": linspace(0, 1e5, 2)
     },
    get_oli_names=True,
    file_name="test_survey",
)

In [ ]:
# individual sample points can be accessed:
survey_points = f.get_survey_sample_conditions(survey, range(len(list(survey.values())[0])))
f.write_output(survey_points, "test_survey_points")

# 4. Login to OLI Cloud.

- The following code demonstrates an OLI Cloud login:

In [ ]:
try:
    credential_manager = CredentialManager(
        username="",
        password="",
        root_url="",
        auth_url="",
        access_keys=[],
    )
except (OSError, ConnectionError) as e:
    print(e)
    credential_manager = None

# 5. Create *.dbs File and 6. Get Output

In [ ]:
if credential_manager:
    with OLIApi(credential_manager) as oliapi:
        # create a new DBS file
        dbs_file_id = oliapi.get_dbs_file_id(
            state_vars["components"],
            thermo_framework="MSE (H3O+ ion)",
            private_databanks=["XSC"],
            phases=["liquid1", "solid"],
            model_name="test",
        )
        
        # get a summary of the DBS file
        file_summary = oliapi.get_dbs_file_summary(dbs_file_id)
        
        # save chemistry information for use in other systems
        chemistry_info = file_summary["chemistry_info"]
        f.write_output(chemistry_info["result"], "chemistry_info")
        
        # create water analysis input
        initial_input = f.build_flash_calculation_input(
            "wateranalysis",
            state_vars,
            file_name="test_water_analysis_input",
        )    
        # run Water Analysis flash calculation survey as specified        
        f.run_flash(
            "wateranalysis",
            oliapi,
            dbs_file_id,
            initial_input,
            survey,
            file_name="test_survey_output"
        )

# 7. Extract Filtered Output

 - OLI's output is robust, so WaterTAP enables printing selected results:

In [ ]:
# extract properties from OLI flash survey output
properties = [
    "prescalingTendencies",
    "entropy",
    "gibbsFreeEnergy",
    "selfDiffusivities",
    "molecularConcentration",
    "kValuesMBased",
]

f.extract_properties(
    survey_output,
    properties,
    samples=range(4,8),
    file_name="test_survey_output_extract",
)